In [1]:
import numpy as np
import pandas as pd

In [2]:
def linear(x, weights, bias):
    return np.dot(x, weights) + bias

def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

# Prepare training data
Load csv files

In [33]:
# I stored the csv files in a folder called "data"
xor_file = "data/xor.csv"
or_file = "data/or.csv"
and_file = "data/and.csv"
nor_file = "data/nor.csv"
nand_file = "data/nand.csv"

Read csv files using panda

In [4]:
xor_data = pd.read_csv(xor_file).values
or_data = pd.read_csv(or_file).values
and_data = pd.read_csv(and_file).values
nor_data = pd.read_csv(nor_file).values
nand_data = pd.read_csv(nand_file).values

Split into input and labels

In [5]:
xor_input = xor_data[:,0:2]
xor_label = xor_data[:,2]
or_input = or_data[:,0:2]
or_label = or_data[:,2]
and_input = and_data[:,0:2]
and_label = and_data[:,2]
nor_input = nor_data[:,0:2]
nor_label = nor_data[:,2]
nand_input = nand_data[:,0:2]
nand_label = nand_data[:,2]

Combining into arrays 

In [6]:
training_inputs = [xor_input, or_input, and_input, nor_input, nand_input]
training_labels = [xor_label, or_label, and_label, nor_label, nand_label]

# Helper functions

In [7]:
# Fetch index in training_inputs and training_labels for current logic gate
def class_index(logic_gate):
    if logic_gate == "xor":
        return 0
    elif logic_gate == "or":
        return 1
    elif logic_gate == "and":
        return 2
    elif logic_gate == "nor":
        return 3
    elif logic_gate == "nand":
        return 4
    else:
        # If all, or unvalid input, return -1, resulting in training on all gates
        return -1

# Model
Used the MLP code as reference

In [24]:
class Model():
    def __init__(self, num_features, num_hidden, num_output):
        # Number of features
        self.num_features = num_features 
        # Number of hidden layers
        self.num_hidden = num_hidden
        #Number of classes
        
        # Hidden layer weight matrix
        # rows=neurons in hidden layer = num_hidden
        # cols=neurons in prev layer = num_features
        self.weights_hidden = np.zeros((num_hidden, num_features), dtype = float)
        # Hidden layer bias
        self.bias_hidden = np.zeros(num_hidden, dtype= float)
        
        # Output layer weight matrix
        # rows=neurons in output = num_output
        # cols=neurons in hidden layer = num_hidden
        self.weights_output = np.zeros((num_output, num_hidden), dtype= float)
        # Output bias
        self.bias_output = np.zeros((num_output), dtype= float)
        
    def predict(self, x):
        # Linear classifier on hidden layer
        linear1 = linear(x, np.transpose(self.weights_hidden), self.bias_hidden)
        # Wrapping output from hidden layer in sigmoid function
        output1 = sigmoid(linear1)
        
        # Linear classifier on output from hidden layer
        linear2 = linear(output1, np.transpose(self.weights_output), self.bias_output)
        # Wrapping final output in sigmoid function
        output2 = sigmoid(linear2)
            
        return output1, output2
    
    def mean_square_error(self, x, y):
        # using the Mean-Squares method, implemented using numpy
        
        # output vs expected
        diff =  np.subtract(y,x)
        
        # square difference and take the mean
        mean_of_square = np.square(diff).mean()
        
        return mean_of_square
    
    def sigmoid_derivative(self, x):
        # Calculating the sigmoid derivative of x using method from MLP code
        return x*(1-x)
    
    def backwards_propagation(self, x, out1, out2, y):
        # Gradient for hidden to output?????????????????
        loss_out = self.mean_square_error(out2,y) * self.sigmoid_derivative(out2)
        # Gradient for weight, hidden to output layer
        loss_out_w = np.dot(np.transpose(loss_out), out1)
        # Gradient for bias, hidden to output layer
        loss_out_b = np.sum(loss_out)
       
        
        # Gradient for inout to hidden???????????????????
        loss_hid = np.dot(loss_out, self.weights_output) * self.sigmoid_derivative(out1)
         # Gradient for weight, input to hidden layer
        loss_hid_w = np.dot(np.transpose(loss_hid), x)
        # Gradient for bias, input to hidden layer
        loss_hid_b = np.sum(loss_hid)
        
        return loss_out_w, loss_out_b, loss_hid_w, loss_hid_b
    
    def train(self, x, y, lr):
        
        #Predicting output 1 and 2
        output1, output2 = self.predict(x)
      
        # Calculating errors
        loss_out_w, loss_out_b, loss_hid_w, loss_hid_b = self.backwards_propagation(x, output1, output2, y)
        
        # == Update weights ==
        #Adjusting weight for hidden layer
        self.weights_hidden -= lr*loss_hid_w
        #Adjusting bias for hidden layer
        self.bias_hidden -= lr*loss_hid_b
        #Adjusting weight for output
        self.weights_output -= loss_out_w
        #Adjusting bias for output
        self.bias_output -= loss_out_b

# Init

In [27]:
# == Initiating model ==
# num_features = 2. x1 and x2
# num_hidden = 3. Feels like a good number when we only have 2 features
# num_output = 1. Binary value between 0 and 1
model = Model(num_features=2, num_hidden=3,num_output=1)

# Learning rate = 0.01. Choosing 0.01 to start, could probably be optimized
LEARNING_RATE = 0.01

# Number of times to run training loop. Choosing 10 since our data is so small
TRAINING_ROUNDS = 10

# What logic gate that will be trained. xor, or, and, nand, nor, all of them
LOGIC_GATE = "all"
# Fetching the index for the choosen logic gate
INDEX = class_index(LOGIC_GATE)

# Train

In [30]:
# Train the model
def run_training(index):
    for training_rounds in range(TRAINING_ROUNDS):
        model.train(training_inputs[index], training_labels[index], lr=LEARNING_RATE)          

In [32]:
if INDEX == -1:
    # Train model with all gates
    for gate in range(5):
        run_training(gate)
else:
    # Train model with specified gate
    run_training(INDEX)